Code to demonstrate basic tetrad_plus capabilities.

In [ ]:
from fastcda import FastCDA
from dgraph_flex import DgraphFlex
import semopy
import pprint as pp

# create  an instance of FastCDA
fc = FastCDA()


In [ ]:
# read in the sample ema dataset and view it
df = fc.getEMAData()
df

In [ ]:
# add the lags, with a suffix of '_lag'
df_lag = fc.add_lag_columns(df, lag_stub='_lag')
df_lag

In [ ]:
# standardize the data
df_lag_std = fc.standardize_df_cols(df_lag)
df_lag_std

In [ ]:
# Create the knowledge prior content for temporal
# order. The lag variables can only be parents of the non
# lag variables
knowledge = {'addtemporal': {
                            0: ['alcohol_bev_lag',
                                'TIB_lag',
                                'TST_lag',
                                'PANAS_PA_lag',
                                'PANAS_NA_lag',
                                'worry_scale_lag',
                                'PHQ9_lag'],
                            1: ['alcohol_bev',
                                'TIB',
                                'TST',
                                'PANAS_PA',
                                'PANAS_NA',
                                'worry_scale',
                                'PHQ9']
                            }
            }

In [ ]:
# run model with run_model_search
result, _ = fc.run_model_search(df_lag_std, 
                             model = 'gfci',
                             score={'sem_bic': {'penalty_discount': 1.0}},
                             test={"fisher_z": {"alpha": .01}},
                             knowledge=knowledge
                             )


In [ ]:
# show the edges that were found
result['edges']

In [ ]:
# create a graph
dg = DgraphFlex()  # create an object
dg.add_edges(result['edges'])  # add edges
dg.show_graph()   # show the graph

In [ ]:
# we can also store the graph in high resolution
# in png format. A graphviz dot file is also generated
# which can be directly edited and rendered using the dot command
# from graphviz
dg.save_graph('boston_graph')

In [ ]:
# Now lets run the sem to get edge weights and
# the signs.  We use the semopy package.
# First we create the model description in
# lavaan format  
# convert the edges to lavaan format
lavaan_model = tp.edges_to_lavaan(result['edges'])

pp.pprint(lavaan_model)


In [ ]:
# run semopy with the lavaan model and the data
sem_results = fc.run_semopy(lavaan_model, df_lag_std)


In [ ]:
# the main output we are interested in is the parameter estimates
sem_results['estimates']

In [ ]:
# save the graph generated by semopy into a png file
png_path = 'boston_data.png'
g = semopy.semplot(sem_results['model'], png_path,  plot_covs = True)


In [ ]:
# add the sem results to our graph object

fc.add_sem_results_to_graph(dg, sem_results['estimates'])
# save the graph to a png file
dg.save_graph('boston_graph_single_sem')

# show the graph
dg.show_graph()